# Crawl Data Twitter 
The crawling process was done using Tweet-Harvest



In [1]:
#@title Twitter Auth Token

twitter_auth_token = 'GG_GEMINK'

In [2]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

Get:1 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]
Get:2 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [4557 B]
Get:4 https://dl.yarnpkg.com/debian stable/main all Packages [11.8 kB]         
Get:5 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.8 kB]       
Get:6 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [293 kB]
Get:8 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2714 B]
Get:9 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]     
Get:10 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease [28.0 kB] 
Get:12 http://archive.ubuntu.com/ubuntu 

In [5]:
!npm install -g npm@10.8.1

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
removed 2 packages, and changed 63 packages in 4s
⠏
⠏24 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
!curl -L -o 'data tourism.xlsx' "https://docs.google.com/spreadsheets/d/1ealGhVIkeDGOoAGTbsml_SHqjIQaie_x2sssE8k1ISM/export?format=xlsx"

In [ ]:
import pandas as pd

file_path = 'data tourism.xlsx'
df_T = pd.read_excel(file_path, sheet_name="tourism_with_id")
df_T

In [ ]:
tgl = [
    "2023-06-01",
    # "2023-07-01",
    "2023-08-01",
    # "2023-09-01",
    "2023-10-01",
    # "2023-11-01",
    "2023-12-01",
    # "2024-01-01",
    "2024-02-01",
    # "2024-03-01",
    "2024-04-01",
    # "2024-05-01",
    "2024-06-01"
]

In [ ]:
# Crawl Data
import glob
import os
dir = 'tweets-data'
def crawl_data(place_id, judul):
  raw = judul
  parts = raw.split('(')
  tempat_wisata = parts[0].strip()
  if len(parts) > 1:
      AKA = parts[1].rstrip(')')
  temp=tempat_wisata
  temp = temp.replace(" ","_")
  for i in range(len(tgl)-1):
    filename = '{}_{}.csv'.format(temp,i)
    src= AKA if len(parts) > 1 else tempat_wisata
    search_keyword = '{} since:{} until:{} lang:id Min_faves:1'.format(src,tgl[i],tgl[i+1])
    limit = 5
    print(search_keyword)
    !npx -y tweet-harvest@2.6.1 -o "{filename}" -s "{search_keyword}" -l {limit} --token {twitter_auth_token}
  dfs = []
  # print(temp)
  csv_files = glob.glob(os.path.join(dir,'{}*.csv'.format(temp)))
  # print(csv_files)
  for file in csv_files:
    try:
        df = pd.read_csv(file)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"File {file} is empty and will be skipped.")
    finally:
        os.remove(file)
  if dfs:
    merged_df=pd.concat(dfs,ignore_index=True)
    merged_df=merged_df[['created_at','favorite_count','full_text','quote_count','reply_count','retweet_count']]
    merged_df.to_csv('{}_{}.csv'.format(place_id,temp),index=False)

In [ ]:
first = 5
last = 6
for i in range(first,last+1):
  rows = df_T[df_T['Place_Id'] == i]
  for index, row in rows.iterrows():
      crawl_data(i,row['Place_Name'])

In [ ]:
from google.colab import files
import zipfile
import os

csv_files = glob.glob('*.csv')
zip_filename = '{}_{}.zip'.format(first,last)
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in csv_files:
        zipf.write(file)
        os.remove(file)
files.download(zip_filename)